In [1]:
import os
import json
import random
from datetime import datetime
import tempfile
import base64
from pathlib import Path

import glob
import json

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src/functionapp")
#On windows use  sys.path.append(module_path+"\\src\\functionapp")

from ai_ocr.azure.openai_ops import load_image, get_size_of_base64_images
from ai_ocr.azure.images import convert_pdf_into_image
from ai_ocr.model import Config
from ai_ocr.chains import get_structured_data
from ai_ocr.azure.doc_intelligence import get_ocr_results

from langchain_core.output_parsers.json import parse_json_markdown

from dotenv import load_dotenv
load_dotenv()

True

### Run the Solution once on the demo to produce an output.json

In [3]:
input_directory = '../demo/medical-dataset/'
#input_directory = '../demo/default-dataset/'
#input_directory = '../demo/eval-dataset/'

system_prompt =  ''
with open(input_directory+'system_prompt.txt', 'r') as file_sys_prompt:
    system_prompt = file_sys_prompt.read()

output_schema = ''
with open(input_directory+'output_schema.json', 'r') as file_output_schema:
    output_schema = file_output_schema.read()

# Create a dict with content key to store the OCR results
ocr_result = {
    "content": ""
}

# Loop over directory and process all PDFs
for file in os.listdir(input_directory):
    if file.endswith(".pdf"):
        ocr_result["content"] += get_ocr_results(input_directory+file).content
        print(f"OCR results:{ocr_result['content'] }")

        # Extract images from the PDF
        convert_pdf_into_image(input_directory+file)
    
 # Ensure the /tmp/ directory exists
imgs_path = "/tmp/"
os.makedirs(imgs_path, exist_ok=True)
    
# Determine the path for the temporary images
imgs = glob.glob(f"{imgs_path}/page*.png")
    
# Limit images by config
config = Config()
print(f"Config img size: {config.max_images}")
imgs = imgs[:config.max_images]
imgs = [load_image(img) for img in imgs]
print(f"Images count: {len(imgs)}")

# Check and reduce images total size if over 20MB
#max_size = 20 * 1024 * 1024  # 20MB
#while get_size_of_base64_images(imgs) > max_size:
#    imgs.pop()
    
# Get structured data
structured = get_structured_data(ocr_result["content"], system_prompt, output_schema, imgs)
    
# Parse structured data and return as JSON
x = parse_json_markdown(structured.content)  
response = json.dumps(x)

print(f'Response: {response}')

actual_output_path = "/tmp/output.json"
with open(actual_output_path, 'w') as f:
    f.write(response)

OCR results:Dottore di Ricerca Responsabile Chirurgia Oculistica Clinica Rugani - Siena Tel. 335-8118324 0577-270652 (ore 20)

VALUTAZIONE PER CHIRURGIA REFRATTIVA
===

Dati anamnestici:

stabilità refrattiva dal 1997.

Aloni notturni

\+ con occhiali Farmaci assunti

Tolleranza lenti a contatto

Buona

Diabete

Secchezza oculare

Mal. del collagene

\-

Autorefrattometro: OD

\-6,7-1,7€ 180

OS - 7,7- 1,5 a 175

Visus OD

\-7-2,25 € 180

Visus OS

\-7,25 -2 a 175

Anomalie ortottiche:

Corneal map:

||||
| - | - | - |
| Schirmer tear test: || |
| BUT: || |
| Pupillometria scotopica: Pachimetria centrale OD | 8 
|| ||| 555 OS 553  |
| Cornea Regol || |
| Cristallino N || |
| Fundus 2 || |
| Tonometria 16 || |
| Anomalie palpebrali e congiuntivali || |

a.19

23.10.02

Specialista in Oculistica Dottore di Ricerca Responsabile Chirurgia Oculistica Clinica Rugani - Siena

Tel. 335-8118324

0577-270652 (ore 20)

DECORSO POST-LASER AD ECCIMERI - Sig.

<figure>

![](figures/0)

</figure>


N

## 1. Evaluating with the LLM

### Load the input (an output from previous LLM run) and evaluate using LLM as a judge

In [5]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

from ai_ocr.azure.openai_ops import get_llm

with open(actual_output_path, 'r') as f:
    input = f.readlines()

#print(input)

system_message = f"""

         You are the OpenAI model that can extract and describe images provided by the user in extreme detail. 
         The user attached images to this message for you to analyse, there is MOST DEFINITELY an image attached, you will never reply saying that you cannot see the image because the image is absolutely and always attached to this message.
         
         Your tasks are the following:
         1. Use the images provided to extract all the data using this customs instructions:
            ```
            {system_prompt}
            ```
            and fit the data following the provided JSON schema:
            ```
            {output_schema}
            ```
            No need to output this extraction in your response.

         2. Compare the data provided in the json input fields against what you have extracted from the images at task 1.

            For each individual field in the json input, how confident you are that what you've extracted from the images field is accurate compared to the same field in the json input? Assign a confidence score between 0 and 1, using the following guidelines:
            - 1.0: Perfect match, absolutely certain
            - 0.9-0.99: Very high confidence, but not absolutely perfect
            - 0.7-0.89: Good confidence, minor uncertainties
            - 0.5-0.69: Moderate confidence, some discrepancies or uncertainties
            - 0.3-0.49: Low confidence, significant discrepancies
            - 0.1-0.29: Very low confidence, major discrepancies
            - 0.0: Completely incorrect or unable to verify
 
            Be critical in your evaluation. It's extremely rare for fields to have perfect confidence scores. If you're unsure about a field assign a lower confidence score.

            Your final response is a json object like the following:
 
            {{
                "field_name": {{
                    "input_value": input_value,
                    "image_value": extracted_value,
                    "confidence": confidence_score,
                }},
                ...
            }}

    
         3. Create an object named "accuracyStatistics" at the end of the json that contains these 4 fields:
          - "numberOfFieldsSeenInImages" is the total number of fields you extracted from the images (for all confidence score);
          - "numberofFieldsInSchema"  is the total number of fields in the provided JSON schema (excluding parent objects for all confidence score);
          - "percentagePresenceAccuracy" which is the ratio between the total fields in the provided JSON schema and the ones extracted from the images;
          - "overallFieldAccuracy" is the sum of the confidence score you gave for each field in percentage.
                  
         ..and take your time to complete the tasks, don't rush.
   
        """

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_message),
        HumanMessagePromptTemplate.from_template("Here is the json input fields:\n{input}"),
    ]
)

messages = chat_template.format_messages(input=input)

if imgs:
    messages.append(HumanMessage(content="Here are the images from the document:"))
    for img in imgs:
        messages.append(HumanMessage(content=[{"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img}"}}]))

model = get_llm()
response = model.invoke(messages)

print(response.content)


```json
{
  "doctor": {
    "specialty": {
      "input_value": "Oculistica",
      "image_value": "Oculistica",
      "confidence": 1.0
    },
    "name": {
      "input_value": "Dottore di Ricerca",
      "image_value": "Dottore di Ricerca",
      "confidence": 1.0
    },
    "clinic": {
      "input_value": "Clinica Rugani - Siena",
      "image_value": "Clinica Rugani - Siena",
      "confidence": 1.0
    },
    "phone": {
      "input_value": "335-8118324, 0577-270652 (ore 20)",
      "image_value": "335-8118324, 0577-270652 (ore 20)",
      "confidence": 1.0
    },
    "fax": {
      "input_value": "",
      "image_value": "",
      "confidence": 1.0
    }
  },
  "patient": {
    "name": {
      "input_value": "",
      "image_value": "N/A",
      "confidence": 0.0
    }
  },
  "post_surgery_follow_up": {
    "period": {
      "input_value": "",
      "image_value": "N/A",
      "confidence": 0.0
    },
    "date": {
      "input_value": "26.10.09",
      "image_value": "26.10.09

In [ ]:
# Delete all generated images created after processing
for file in os.listdir(imgs_path):
    if file.endswith(".jpeg") or file.endswith(".png"):
        image_path = os.path.join(imgs_path, file)
        try:
            os.remove(os.path.join(imgs_path, file))
            print(f"Deleted image: {image_path}")
        except Exception as e:
            print(f"Error deleting image {image_path}: {e}")

## 2. Evaluating with ground truth

### Load the input (an output from previous LLM run), ground truth and create a jsonl file

In [ ]:
import sys
if module_path not in sys.path:
    sys.path.append(module_path)

import json
import time
from pprint import pprint

def compile_jsonl(ground_truth_path, actual_output_path, output_file):
    # Read the ground truth JSON file
    with open(ground_truth_path, 'r') as gt_file:
        ground_truth = json.load(gt_file)

    with open(eval_schema_path, 'r') as eval_schema_file:
        eval_schema = json.load(eval_schema_file)


    def merge_dicts(ground_truth, eval_schema):
        merged_dict = {}
        
        for key, value in ground_truth.items():
            if isinstance(value, dict):
                merged_dict[key] = merge_dicts(value, eval_schema.get(key, {}))
            elif isinstance(value, list):
                merged_dict[key] = {
                    "value": value,
                    "eval": eval_schema.get(key, [])
                }
            else:
                merged_dict[key] = {
                    "value": value,
                    "eval": eval_schema.get(key, [])
                }
        
        return merged_dict

    # Open the output file
    with open(output_file, 'w') as out_file:
        # Iterate over each actual output JSON file
        with open(actual_output_path, 'r') as af:
            actual_data = json.load(af)
            # Combine ground truth and actual data into one object
            combined_data = {"ground_truth": ground_truth, "actual": actual_data, "eval_schema":eval_schema}
            # Write the combined data as a single line in the jsonl file
            out_file.write(json.dumps(combined_data) + '\n')


ground_truth_path = f"{module_path}/demo/default-dataset/ground_truth.json"
eval_data_path = f"{module_path}/demo/default-dataset/eval_data.jsonl"
eval_schema_path = f"{module_path}/demo/default-dataset/evaluation_schema.json"

compile_jsonl(ground_truth_path, actual_output_path, eval_data_path)



### Evaluate using ground truth

In [ ]:
from promptflow.evals.evaluate import evaluate
from src.evaluators.json_evaluator import JsonEvaluator
eval_data_path = f"{module_path}/demo/default-dataset/eval_data.jsonl"
with open(eval_data_path) as file:
    data = json.load(file)
    ground_truth = data["ground_truth"]
    evaluation_schema = data["eval_schema"]

evaluators = {}
evaluator_config = {}
default_match_evaluator_config = {}
json_evaluator = JsonEvaluator()
evaluators["json_evaluator"] = json_evaluator
evaluator_config["json_evaluator"] = {
    "actual": "${data.actual}",
    "ground_truth": "${data.ground_truth}",
    "eval_schema": "${data.eval_schema}"
}

timestamp = time.strftime("%m_%d.%H.%M.%S")
output_path = f"{module_path}/notebooks/outputs/output_{timestamp}.json"

results = evaluate(
    evaluation_name="test_eval_1",
    data=eval_data_path,
    evaluators=evaluators,
    evaluator_config=evaluator_config,
    output_path=output_path
)
pprint(results)
